### 【問題1】公式Exampleを分担して実行¶
TensorFLowの公式Exampleを分担して実行してください。
以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。

In [1]:
import tensorflow as tf
from keras import backend as K
from keras.datasets import mnist

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

C:\Users\anai\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


tutorials → Build a Convolutional Neural Network using Estimators

このexampleはtensorflowを使って2次元のCNNを作成している
以下サンプルコードとコードリーティング結果を示す

In [2]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)  

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

データの整形や畳み込み層などをテンソルとして保存している
結果をpredictionsという辞書を用いて格納している
dropoutを指定し過学習を防いでいる

In [3]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

tf.estimator.Estimatorを使ってモデルのインスタンスを作成している
model_fn引数に先ほどの関数を指定することで、任意のモデルを作成している

In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

データをestimator.inputs.numpy_input_fnで準備
先ほど作成したクラス.trainで学習している
input_fnには準備したデータを入力している

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

validationデータを準備している
先ほどのクラス.evaluateで推定している

実行結果(google colaboratoryで実行しました)
INFO:tensorflow:Calling model_fn. INFO:tensorflow:Done calling model_fn. INFO:tensorflow:Starting evaluation at 2019-09-15T06:23:32Z INFO:tensorflow:Graph was finalized. INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-2002 INFO:tensorflow:Running local_init_op. INFO:tensorflow:Done running local_init_op. INFO:tensorflow:Finished evaluation at 2019-09-15-06:23:39 INFO:tensorflow:Saving dict for global step 2002: accuracy = 0.848, global_step = 2002, loss = 0.6666925 INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2002: /tmp/mnist_convnet_model/model.ckpt-2002 {'accuracy': 0.848, 'loss': 0.6666925, 'global_step': 2002}

最終的な正解率および損失
{'accuracy': 0.8551, 'loss': 0.6025994, 'global_step': 2002}

### 問題2】Iris（2値分類）をKerasで学習¶
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [6]:
"""
th.kerasで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
#dataset_path = r"Iris.csv"
df = pd.read_csv(r"C:\Users\anai\dive\Dataset\iris\iris.csv")
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

input_data = tf.keras.layers.Input(shape=(n_input,))
x = tf.keras.layers.Dense(n_hidden1, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(n_hidden2, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(n_classes, activation=tf.nn.sigmoid)(x)

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=2,
                   validation_data=(X_val, y_val))

y_pred_proba = model.predict(X_test)[:, 0]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

#print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)
#結果がいらず、評価のみ行う場合はevaluateメソッドも便利です。
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W1030 11:14:52.860879 17208 deprecation.py:506] From C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1030 11:14:52.978695 17208 deprecation.py:323] From C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 50)                250       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 5,451
Trainable params: 5,451
Non-trainable params: 0
_________________________________________________________________
Train on 64 samples, validate on 16 samples
Epoch 1/10
64/64 - 0s - loss: 0.9593 - acc: 0.4688 - val_loss: 0.8827 - val_acc: 0.3750
Epoch 2/10
64/64 - 0s - loss: 0.7812 - acc: 0.5312 - val_loss: 0.7097 - val_acc: 0.3750
Epoch 3/10
64/64 - 0s - loss: 0.6515 

### 【問題3】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [7]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
"""
from sklearn.preprocessing import OneHotEncoder
import sklearn.preprocessing as sp

# データセットの読み込み
#dataset_path = r"Iris.csv"
df = pd.read_csv(r"C:\Users\anai\dive\Dataset\iris\iris.csv", delimiter=',')
# データフレームから条件抽出
#df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df['Species']
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

#onehot
#enc = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype='int', categories='auto')
enc = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype='int')
y_1hot = enc.fit_transform(y[:, np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_1hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3


input_data = tf.keras.layers.Input(shape=(n_input,))
x = tf.keras.layers.Dense(n_hidden1, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(n_hidden2, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)(x)

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=2,
                    validation_data=(X_val, y_val))

y_pred_proba = model.predict(X_test)[:,:]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)
#結果がいらず、評価のみ行う場合はevaluateメソッドも便利です。
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                250       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 5,653
Trainable params: 5,653
Non-trainable params: 0
_________________________________________________________________
Train on 96 samples, validate on 24 samples
Epoch 1/10
96/96 - 0s - loss: 1.0413 - acc: 0.4792 - val_loss: 0.6270 - val_acc: 0.7083
Epoch 2/10
96/96 - 0s - loss: 0.5178 - acc: 0.7917 - val_loss: 0.3977 - val_acc: 0.7917
Epoch 3/10
96/96 - 0s - loss: 0.361

### 【問題4】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [8]:
"""
TensorFlowで実装したニューラルネットワークを使いHouse Pricesデータセットの価格を推定する。
"""
from sklearn.preprocessing import StandardScaler

# データセットの読み込み
df = pd.read_csv(r"C:\Users\anai\dive\Dataset\House Prices\train.csv", delimiter=',')
#df_test = pd.read_csv(r"C:\Users\anai\dive\Dataset\House Prices\test.csv", delimiter=',')

# データフレームから条件抽出
y = df['SalePrice']
X = df.loc[:,["GrLivArea", "YearBuilt"]]

y = np.array(y)
X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

#std = StandardScaler()
#X_train = std.fit_transform(X_train)
#X_test = std.fit_transform(X_test)
#X_val = std.fit_transform(X_val)

y_test = y_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

# ハイパーパラメータの設定
learning_rate = 0.00000000001
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

########

input_data = tf.keras.layers.Input(shape=(n_input,))
x = tf.keras.layers.Dense(n_hidden1, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(n_hidden2, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(n_classes)(x)

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='mean_squared_error',
              optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate),
              metrics=['mean_squared_error'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=2,
                    validation_data=(X_val, y_val))

y_pred_proba = model.predict(X_test)[:,:]

# 確率を0, 1に変換
#y_pred = np.where(y_pred_proba >0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
#print("y_pred", y_pred)
#結果がいらず、評価のみ行う場合はevaluateメソッドも便利です。
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                150       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 5,351
Trainable params: 5,351
Non-trainable params: 0
_________________________________________________________________
Train on 934 samples, validate on 234 samples
Epoch 1/10
934/934 - 0s - loss: 37206157114.6552 - mean_squared_error: 37206171648.0000 - val_loss: 28063671348.5128 - val_mean_squared_error: 28063674368.0000
Epoch 2/10
934/934 - 0s - loss: 9594072866.6724 -

### 【問題5】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [9]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTデータセットを3値分類する
"""
# データセットの読み込み
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 画像データを2次元に変換
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

#onehot
enc = Oneenc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.fit_transform(y_test[:, np.newaxis])


# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.05
batch_size = 100
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10


input_data = tf.keras.layers.Input(shape=(n_input,))
x = tf.keras.layers.Dense(n_hidden1, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(n_hidden2, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)(x)

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=100,
                    epochs=10,
                    verbose=2,
                    validation_data=(X_val, y_val))

y_pred_proba = model.predict(X_test)[:,:]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

#print("y_pred_proba", y_pred_proba)
#print("y_pred", y_pred)
#結果がいらず、評価のみ行う場合はevaluateメソッドも便利です。
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                39250     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1010      
Total params: 45,360
Trainable params: 45,360
Non-trainable params: 0
_________________________________________________________________
Train on 38400 samples, validate on 9600 samples
Epoch 1/10
38400/38400 - 1s - loss: 0.2990 - acc: 0.9090 - val_loss: 0.2055 - val_acc: 0.9364
Epoch 2/10
38400/38400 - 1s - loss: 0.1604 - acc: 0.9504 - val_loss: 0.1631 - val_acc: 0.9502
Epoch 3/10
38400